## 摘要
我们的方法通过Sleep-WAke网络（SWAN）明确地建模输出序列中的短语结构。SWAN是近期提出的一种基于分割的序列模型方法。为了减轻SWAN所需的单调的对齐，我们提出了一种新的层来完成输入序列的局部重排序。

## 导语
人类语言总是表现出强大的组合模式。例如machine learning is part of artificial intelligence，我们可以把他拆解成[machine learning] [is] [part of] [of] [artificial intelligence]。我们称每个框内的内容为短语，短语有自己的含义，在其他语境中也能被重复使用。<br>
在此论文中，我们开发了一种NMT能清楚地在输出语言上建模短语。传统基于短语的统计机器学习方法已经表明这种方法超越了基于单词的方法(Koehn et al. 2013; Koehn, 2009; Lopez, 2008)。<br>
NPMT建立与Sleep-WAke网络之上，这是一种基于分割的序列建模技术。在本文中，分割部分被作为目标序列中的短语。然而，SWAN的输入与输出之间需要繁琐的校准，在机器翻译中是不这样的。为了解决这个问题，我们再SWAN下面引入了一个新层，能在输入序列上完成局部重新排序。<br>
![NPMT](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/NPMT.png)

## Neural Phrase-based machine translation
### Modeling phrases with SWAM
在隔壁
### Local Reordering of input sequences
SWAN假设输入的元素和输出的分组之间存在单调的对应关系（不存在交叉对应）。SWAN assumes a monotonic alignment between the output segments and the input elements.<br>
这对于语音识别的任务来说，是可以理解的。但是，对于MT来说，就有点太严格了。<br>
在神经网路中，一般使用Attention机制来处理对齐（alignment）任务，然而在SWAN中引入Attention的方法，现在还并不明确<br>
应该提到的是，使用bidirectional RNN encoder可以一定程度上减缓对齐问题，然而这并不够。<br>
在这里，我们不使用Attention，而是引入一个reorder层。不是用Attention的一个好处是，他可以大大的加快decoder的工作效率。<br>
下面开始介绍reordering层：
- 使得输入为$e_{1:T'}$，输出为$h_{1:T'}$，我们通过下面的方式计算$h_t$:
- $h_t=tanh(\sum_{i=1}^{\tau}\sigma(w_i^T[e_{t-\tau+1};...;e_t])e_{t-r+i})$
- 其中$\sigma$是一个sigmoid function，$\tau$是局部reordering窗口的大小。[]代表着链接（concatenation）。sigmoid部分在这里相当于一个gate，其值由当前窗口中的输入决定（对应观察对象到当前时间点的输入）。最终的输出是窗口内输入的加权求和，然后再通过非线性方程tanh()求得。
- 有点像Attention，把当前时间的输入映射为对应gate比较大的时间点的输入。
- 因为我们不需要Query，因此相较于Attention，我们有两点不同。We design our layer differently from the typical attention mechanisn in two ways because we do not have a query to begin with as in standard attention mechanisms.
    - We do not normalize the weights for the input elements $e_{t-\tau+1},e_{t-\tau},..,e_t$. This provides the reordering capability and can shut off everything it needed.
    - The weight of any position i in the reordering window is determined by all input elements $e_{t-\tau+1},e_{t-\tau},..,e_t$ in the window
![reordering](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/reordering.png)

### 并不是很理解“基于短语”表现在哪里？？？

One other related work to this layer is the Gated Linear Units(GLU) which can control the information flow of the output of a traditional convolutional layer. But GLU does not havethe ability to choose which input elements from the convolution window. And in our experiments, we found neither GLU nor traditional convolutional layer helped our setup of using SWAN.